In [1]:
ratings= sqlContext.read.csv('/FileStore/tables/qthksaj91493514788924/rating_cleaned.csv', sep=',' , inferSchema=True, header=True)
anime = sqlContext.read.csv('/FileStore/tables/ezjvoa7k1493514682679/anime_clean.tsv', sep='\t', inferSchema=True, header=True)
names= sqlContext.read.csv('/FileStore/tables/p4kjul861493612915497/anime_name.csv', inferSchema=True, header=True)

In [2]:
print ratings.show(5)
print anime.show(5)
print names.show(5)

In [3]:
ratings=ratings.filter(ratings.rating!=-1)
print ratings.show()

In [4]:
(training, test) = ratings.randomSplit([0.8, 0.2])
print(training.count(), test.count())

In [5]:
from pyspark.ml.recommendation import ALS
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="anime_id", ratingCol="rating")
model = als.fit(training)

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator
#als = ALS(maxIter=50,rank=10 ,regParam=0.01, userCol="user_id", itemCol="anime_id",ratingCol="rating") #1.12
#als = ALS(maxIter=25,rank=9 ,regParam=0.01, userCol="user_id", itemCol="anime_id",ratingCol="rating") #1.172
als = ALS(maxIter=25,rank=9 ,regParam=0.01, userCol="user_id", itemCol="anime_id",ratingCol="rating") #1.172
model = als.fit(training)
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
predictions = predictions.na.drop()
rmse = evaluator.evaluate(predictions)

print("Root-mean-square error = " + str(rmse))

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
predictions = predictions.na.drop()
rmse = evaluator.evaluate(predictions)

print("Root-mean-square error = " + str(rmse))

In [8]:
predictions.show(4)

In [9]:
test_user= 21513

In [10]:
rated_movies=test.filter(test.user_id==test_user)

In [11]:
predictions.filter((predictions.user_id==test_user)).sort('prediction', ascending=True).join(names, names.anime_id==predictions.anime_id).show()